<a href="https://colab.research.google.com/github/LoTzuChin/113-1-FinancialBigData/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝 yfinance
!pip install yfinance

In [1]:
!pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.8 MB/s eta 0:00:00


In [21]:
!pip install matplotlib

In [23]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import mplfinance as mpf
from PIL import Image
import io
import matplotlib.pyplot as plt


# 設定股票ID及日期範圍
stock_id = '2330.tw'
end = dt.date.today()
start = end - dt.timedelta(days=365*2)
old_df = yf.download(stock_id, start=start, end=end).reset_index()

# 確保將日期轉換為一維格式
dates = pd.to_datetime(old_df['Date']).dt.tz_localize(None)  # 移除時區資訊

# 創建新的 DataFrame
data = {
    'Open': old_df['Open'].values.flatten(),  # 確保是一維
    'High': old_df['High'].values.flatten(),  # 確保是一維
    'Low': old_df['Low'].values.flatten(),    # 確保是一維
    'Close': old_df['Close'].values.flatten(),# 確保是一維
    'Volume': old_df['Volume'].values.flatten()# 確保是一維
}
df = pd.DataFrame(data, index=dates)

# 定義函數來生成每三天的K線圖圖像數據
def create_image_dataset(df):
    X, y = [], []

    for i in range(len(df) - 3):
        # 提取每三天的 OHLC 數據
        sub_df = df.iloc[i:i+3]
        print(sub_df)
        # 使用mplfinance繪製K線圖，將圖片輸出到內存中
        fig, ax = mpf.plot(
            sub_df,
            type='candle',
            style='charles',
            returnfig=True,
            volume=False
        )

        # 將圖像保存為字節數據並讀入 PIL Image
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        image = Image.open(buf).convert('L')  # 轉為灰度圖以簡化訓練

        # 將圖片大小調整為固定的尺寸 (例如64x64)
        image = image.resize((64, 64))

        # 圖片轉為數組並加入X
        X.append(np.array(image) / 255.0)  # 正規化圖像數據

        # 使用收盤價變化確定標籤
        close_current = df['Close'].iloc[i+3]
        close_previous = df['Close'].iloc[i+2]
        y.append(1 if close_current > close_previous else 0)

        # 關閉圖像以釋放記憶體
        plt.close(fig)

    return np.array(X), np.array(y)

# 創建X和y
X, y = create_image_dataset(df)
X = X.reshape(-1, 64, 64, 1)  # 重塑為CNN所需的形狀 (批次大小, 64, 64, 1)

print("X的形狀:", X.shape)
print("y的形狀:", y.shape)


[*********************100%***********************]  1 of 1 completed


             Open   High    Low  Close    Volume
Date                                            
2022-10-31  386.0  390.0  381.0  390.0  38409054
2022-11-01  388.5  393.0  386.0  391.5  41426496
2022-11-02  391.0  395.0  388.5  395.0  22535650
             Open   High    Low  Close    Volume
Date                                            
2022-11-01  388.5  393.0  386.0  391.5  41426496
2022-11-02  391.0  395.0  388.5  395.0  22535650
2022-11-03  385.0  387.0  383.5  384.0  39731333
             Open   High    Low  Close    Volume
Date                                            
2022-11-02  391.0  395.0  388.5  395.0  22535650
2022-11-03  385.0  387.0  383.5  384.0  39731333
2022-11-04  381.0  384.5  378.5  382.0  28397417
             Open   High    Low  Close    Volume
Date                                            
2022-11-03  385.0  387.0  383.5  384.0  39731333
2022-11-04  381.0  384.5  378.5  382.0  28397417
2022-11-07  390.0  393.0  387.0  390.0  35387706
             Open   

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 使用sigmoid輸出二元分類
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 檢查模型摘要
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 62, 62, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 31, 31, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 29, 29, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 12, 12, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │         294,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 517,121 (1.97 MB)

 Trainable params: 517,121 (1.97 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(X, y, epochs=100, batch_size=8, validation_split=0.2)

Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 8s 141ms/step - accuracy: 0.4556 - loss: 0.7012 - val_accuracy: 0.4583 - val_loss: 0.6937
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 108ms/step - accuracy: 0.5061 - loss: 0.6931 - val_accuracy: 0.5417 - val_loss: 0.6929
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 139ms/step - accuracy: 0.5263 - loss: 0.6930 - val_accuracy: 0.5417 - val_loss: 0.6929
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.5430 - loss: 0.6929 - val_accuracy: 0.5417 - val_loss: 0.6928
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 11s 130ms/step - accuracy: 0.5067 - loss: 0.6932 - val_accuracy: 0.5417 - val_loss: 0.6929
Epoch 6/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 11s 139ms/step - accuracy: 0.5225 - loss: 0.6939 - val_accuracy: 0.4896 - val_loss: 0.6932
Epoch 7/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.5266 - loss: 0.6936 - val_accuracy: 0.5417 - val_loss: 0.6906
Epoch 8/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 11s 129ms/step - accuracy: 0.5473 - loss: 0.6919 - val_a

In [31]:
import io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf

# 生成當前最後三天的K線圖像數據
def get_today_image(df):
    # 提取最後三天的 OHLC 數據
    sub_df = df.iloc[-3:]

    # 使用mplfinance繪製K線圖，並將圖像保存到內存中
    fig, ax = mpf.plot(
        sub_df,
        type='candle',
        style='charles',
        returnfig=True,
        volume=False
    )

    # 保存圖像為字節流
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf).convert('L')  # 轉為灰度

    # 調整圖片大小為64x64
    image = image.resize((64, 64))
    image_data = np.array(image) / 255.0  # 正規化
    plt.close(fig)  # 關閉圖像

    return image_data.reshape(1, 64, 64, 1)

# 獲取今天之前三天的K線圖數據
X_today_image = get_today_image(df)

# 模型預測
prediction = model.predict(X_today_image)
predicted_increase = prediction[0] > 0.5

# 顯示預測結果
if predicted_increase:
    print("預測今日股價將上漲")
else:
    print("預測今日股價將下跌")

# 驗證今天是否真的上漲
if len(df) > 3:  # 確保有今天及前一天的數據
    close_today = df['Close'].iloc[-1]
    close_yesterday = df['Close'].iloc[-2]
    actual_increase = close_today > close_yesterday

    # 檢查模型預測是否正確
    if predicted_increase == actual_increase:
        print("模型預測正確！")
    else:
        print("模型預測錯誤。")
else:
    print("數據不足，無法驗證今日漲跌。")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
預測今日股價將下跌
模型預測正確！
